# 1.创建和使用类
* Python中将一切皆对象，类是创建对象的模板，对象是类的实例化
* OOP是对现实世界的建模，是学习编程模式（或设计模式）和掌握架构思想的基础
* 识别对象，找出共性（属性、行为），抽象为类

In [3]:
class Dog:
    """一次模拟小狗的简单尝试"""

    def __init__(self, name, age):  # __init__作用？ self参数的特点
        """初始化属性 name 和 age"""
        self.name = name
        self.age = age

    def sit(self):
        """模拟小狗收到命令坐下"""
        print(f"{self.name} is now sitting")

    def roll_over(self):
        """模拟小狗收到命令打滚"""
        print(f"{self.name} is now rolling over")


your_dog = Dog("XiaoHei", "5")
your_dog.sit()
your_dog.roll_over()

XiaoHei is now sitting
XiaoHei is now rolling over


在Python中，**MRO（Method Resolution Order，方法解析顺序）** 决定了在多继承场景下，当调用一个方法或属性时，Python解释器搜索该方法的顺序。MRO是Python面向对象编程中非常重要的概念，尤其在多重继承时，它确保了方法调用的确定性和一致性。

---

## **1. 为什么需要MRO？**
在单继承中，方法查找顺序很简单：从子类到父类依次查找。但在多继承中，可能会出现 **“菱形继承”**（Diamond Inheritance）问题，例如：

```python
class A:
    def show(self):
        print("A")

class B(A):
    def show(self):
        print("B")

class C(A):
    def show(self):
        print("C")

class D(B, C):
    pass

d = D()
d.show()  # 输出什么？
```
- `D` 继承自 `B` 和 `C`，而 `B` 和 `C` 又都继承自 `A`。
- 如果调用 `d.show()`，Python 应该先查找 `B.show` 还是 `C.show`？

为了解决这个问题，Python 使用 **C3线性化算法（C3 Linearization）** 来计算 MRO。

---

## **2. Python 的 MRO 规则**
Python 的 MRO 采用 **C3算法**，其核心规则是：
1. **子类优先于父类**（`D` > `B` > `C` > `A`）。
2. **同一层级，按继承顺序从左到右**（`B` 在 `C` 前面，因为 `class D(B, C)`）。
3. **避免重复查找**（每个类在 MRO 链中只出现一次）。

### **MRO 的计算方式**
Python 使用 `类名.__mro__` 或 `类名.mro()` 查看 MRO 顺序：
```python
print(D.__mro__)
# 输出: (<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
```
这意味着：
1. 先查找 `D` 是否有 `show()`。
2. 如果没有，查找 `B` 是否有 `show()`。
3. 如果 `B` 也没有，查找 `C` 是否有 `show()`。
4. 最后查找 `A` 和 `object`。

所以 `d.show()` 会输出 `"B"`，因为 `B` 在 `C` 之前。

---

## **3. MRO 的应用场景**
### **(1) 方法调用顺序**
```python
class A:
    def foo(self):
        print("A")

class B(A):
    def foo(self):
        print("B")
        super().foo()  # 调用 MRO 中的下一个类

class C(A):
    def foo(self):
        print("C")
        super().foo()

class D(B, C):
    pass

d = D()
d.foo()
```
**输出：**
```
B
C
A
```
**解析：**
- `D.__mro__` = `(D, B, C, A, object)`
- `B.foo()` 调用 `super().foo()`，MRO 的下一个类是 `C`，所以执行 `C.foo()`。
- `C.foo()` 又调用 `super().foo()`，MRO 的下一个类是 `A`，所以执行 `A.foo()`。

### **(2) `super()` 的工作原理**
`super()` 不是直接调用父类，而是按照 **MRO 顺序** 调用下一个类：
```python
class A:
    def foo(self):
        print("A")

class B(A):
    def foo(self):
        print("B")
        super(B, self).foo()  # 等同于 super().foo()

class C(A):
    def foo(self):
        print("C")
        super().foo()

class D(B, C):
    pass

d = D()
d.foo()
```
**输出：**
```
B
C
A
```
- `super(B, self).foo()` 会查找 `B` 之后的下一个类（`C`），而不是直接调用 `A`。

---

## **4. MRO 的常见问题**
### **(1) MRO 冲突**
如果继承关系无法线性化（如出现循环继承），Python 会抛出 `TypeError`：
```python
class A(B):
    pass

class B(A):
    pass
```
**报错：**
```
TypeError: Cannot create a consistent method resolution order (MRO) for bases A, B
```

### **(2) 修改 MRO**
MRO 由继承顺序决定，不能直接修改，但可以调整类的继承顺序：
```python
class D(C, B):  # 修改为 C 在前
    pass

print(D.__mro__)  # (<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
```

---

## **5. 总结**
| 关键点               | 说明                        |
|-------------------|---------------------------|
| **MRO 的作用**       | 决定多继承时方法和属性的查找顺序          |
| **MRO 算法**        | C3 线性化（保证子类优先、继承顺序优先）     |
| **查看 MRO**        | `类名.__mro__` 或 `类名.mro()` |
| **`super()` 的行为** | 按照 MRO 顺序调用下一个类，而非直接调用父类  |
| **MRO 冲突**        | 循环继承会导致 `TypeError`       |

**MRO 确保了多继承的确定性和可预测性**，是 Python 面向对象编程的重要机制。理解 MRO 有助于编写更健壮的多继承代码，并避免方法调用的歧义。

# 2.使用类和实例

In [8]:
class Car:
    """一个表示汽车的类"""

    # 类属性（所有实例共享）
    wheels = 4  # 汽车默认有4个轮子

    def __init__(self, make, model, year):
        """初始化汽车属性"""
        self.make = make  # 制造商
        self.model = model  # 型号
        self.year = year  # 年份
        self.odometer_reading = 0  # 里程表读数

    # 实例方法
    def get_descriptive_name(self):
        """返回格式化的完整汽车名称"""
        return f"{self.year} {self.make} {self.model}"

    def read_odometer(self):
        """打印汽车里程"""
        print(f"This car has {self.odometer_reading} miles on it.")

    def update_odometer(self, mileage):
        """
        更新里程表读数
        拒绝回调里程表
        """
        if mileage >= self.odometer_reading:
            self.odometer_reading = mileage
        else:
            print("You can't roll back an odometer!")

    def increment_odometer(self, miles):
        """增加里程表读数"""
        if miles > 0:
            self.odometer_reading += miles
        else:
            print("You can't add negative miles!")


# 创建实例
my_Camry = Car("Toyota", "Camry", 2020)

# 修改属性值的方法
# 方法1: 直接通过修改实例值修改属性值
my_Camry.odometer_reading = 100
print(my_Camry.odometer_reading)

# 方法2: 通过方法修改
# 使用实例方法
my_Camry.increment_odometer(150)
print(my_Camry.odometer_reading)

# 方法3: 通过方法传递（增加特定的值）

100
250


# 3.继承

In [16]:
# 父类 子类
# Car ElectricCar
class ElectricCar(Car):
    """电动汽车的独特之处"""

    def __init__(self, make, model, year):
        """初始化父类的属性"""
        super().__init__(make, model, year)
        """新增属性电池容量并把所有类对象电池容量均默认40kWh"""
        self.battery_size = 40

    def get_descriptive_name(self):
        """重写父类的方法，加上了电池容量"""
        return f"{self.year} {self.make} {self.model} {self.battery_size}"

    def describe_battery(self):
        """打印一条关于电车电池容量的信息"""
        print(f"This car has a {self.battery_size}-kWh battery.")


my_Mustang = ElectricCar("Ford", "Mustang", 2019)
print(my_Mustang.get_descriptive_name())

print(my_Mustang.battery_size)
my_Mustang.describe_battery()

2019 Ford Mustang 40
40
This car has a 40-kWh battery.


## 将代码重构为三个类：Car（父类）、ElectricCar（子类）和 Battery（组合类）。
## 这种设计遵循 "组合优于继承" 的原则，使代码更加模块化和可扩展。

In [ ]:
class Car:
    """一个表示汽车的基类"""

    wheels = 4  # 类属性，所有汽车默认4个轮子

    def __init__(self, make, model, year):
        """初始化汽车属性"""
        self.make = make
        self.model = model
        self.year = year
        self.odometer_reading = 0

    def get_descriptive_name(self):
        """返回格式化的完整汽车名称"""
        return f"{self.year} {self.make} {self.model}"

    def read_odometer(self):
        """打印汽车里程"""
        print(f"This car has {self.odometer_reading} miles on it.")

    def update_odometer(self, mileage):
        """更新里程表读数，防止回调"""
        if mileage >= self.odometer_reading:
            self.odometer_reading = mileage
        else:
            print("You can't roll back an odometer!")

    def increment_odometer(self, miles):
        """增加指定的里程数"""
        if miles > 0:
            self.odometer_reading += miles
        else:
            print("You can't add negative miles!")


class Battery:
    """电动汽车电池的独立类"""

    def __init__(self, battery_size=40):
        """初始化电池属性"""
        self.battery_size = battery_size  # 单位：kWh

    def describe_battery(self):
        """打印电池容量信息"""
        print(f"This car has a {self.battery_size}-kWh battery.")

    def get_range(self):
        """根据电池容量估算续航里程"""
        if self.battery_size == 40:
            range_km = 250
        elif self.battery_size == 60:
            range_km = 315
        else:
            range_km = "unknown"
        print(f"This car can go about {range_km} km on a full charge.")


class ElectricCar(Car):
    """电动汽车类，继承自Car"""

    def __init__(self, make, model, year):
        """
        初始化父类属性
        然后初始化电动汽车特有属性
        """
        super().__init__(make, model, year)
        self.battery = Battery()  # 组合一个Battery实例

    def get_descriptive_name(self):
        """重写父类方法，添加电动标识"""
        return f"{super().get_descriptive_name()} (Electric)"

    def describe_battery(self):
        """委托给Battery类的方法"""
        self.battery.describe_battery()


# 使用示例
if __name__ == "__main__":
    # 测试Car类
    my_camry = Car("Toyota", "Camry", 2020)
    print(my_camry.get_descriptive_name())
    my_camry.increment_odometer(100)
    my_camry.read_odometer()

    # 测试ElectricCar和Battery
    my_tesla = ElectricCar("Tesla", "Model S", 2022)
    print(my_tesla.get_descriptive_name())  # 2022 Tesla Model S (Electric)
    my_tesla.battery.describe_battery()  # This car has a 40-kWh battery.
    my_tesla.battery.get_range()  # This car can go about 250 km...

    # 可以单独配置电池
    big_battery = Battery(60)
    my_tesla.battery = big_battery
    my_tesla.battery.get_range()  # This car can go about 315 km...

# 4.导入类
* 避免在一个文件中包含过多的类

# Python中导入类的相关知识

在Python中，导入类是模块系统的重要组成部分，它允许你使用其他模块中定义的类。以下是关于Python中导入类的详细讲解：

## 基本导入方式

### 1. 导入整个模块

```python
import module_name
```

使用这种方式导入后，需要通过模块名访问其中的类：

```python
import datetime
now = datetime.datetime.now()
```

### 2. 从模块导入特定类

```python
from module_name import ClassName
```

使用这种方式可以直接使用类名而不需要模块前缀：

```python
from datetime import datetime
now = datetime.now()
```

### 3. 导入多个类

```python
from module_name import Class1, Class2, Class3
```

示例：

```python
from collections import Counter, defaultdict, OrderedDict
```

### 4. 导入所有类（不推荐）

```python
from module_name import *
```

这会导入模块中所有公开的类，但通常不推荐使用，因为：
- 可能导致命名冲突
- 代码可读性降低
- 难以追踪类的来源

## 高级导入技巧

### 1. 使用别名

当类名冲突或类名太长时，可以使用别名：

```python
from module_name import LongClassName as LCN
```

示例：

```python
from matplotlib.pyplot import Figure as Fig
```

### 2. 导入子模块中的类

```python
from package.submodule import ClassName
```

示例：

```python
from sklearn.linear_model import LinearRegression
```

### 3. 相对导入（在包内部使用）

在包内部的模块中，可以使用相对导入：

```python
from .sibling_module import ClassName      # 从当前包的兄弟模块导入
from ..parent_module import ClassName     # 从父包导入
```

## 导入时的执行机制

1. **首次导入**：Python会执行模块中的所有顶级代码（包括类定义）
2. **后续导入**：Python会使用已加载的模块，不会重复执行代码
3. **类定义**：导入时会执行类定义块，但不会执行类方法（除非是装饰器或方法定义中的代码）

## 最佳实践

1. **明确导入**：尽量使用`from module import Class`形式，明确知道类来自哪里
2. **避免循环导入**：A模块导入B模块，B模块又导入A模块会导致问题
3. **组织导入顺序**：
   - 标准库导入
   - 第三方库导入
   - 本地应用/库导入
4. **使用`__all__`**：在模块中定义`__all__`列表来控制`from module import *`的行为

## 示例

```python
# 标准库导入
from datetime import datetime
import os

# 第三方库导入
from flask import Flask
import numpy as np

# 本地应用导入
from my_package.utils import HelperClass

# 使用别名
from pandas import DataFrame as DF

class MyClass:
    pass
```

## 动态导入

在需要时可以使用`importlib`动态导入：

```python
import importlib

module = importlib.import_module('module_name')
MyClass = getattr(module, 'ClassName')
```

理解Python中的类导入机制有助于编写更模块化、更可维护的代码，并能有效管理项目依赖关系。

# 5.编程风格

# Python编程风格指南

Python以其优雅、明确和简单的设计哲学而闻名，良好的编程风格是Python开发的重要组成部分。以下是Python编程风格的关键要点：

## 1. PEP 8 - Python官方风格指南

PEP 8是Python的官方风格指南，主要内容包括：

### 代码布局
- **缩进**：使用4个空格（不用制表符Tab）
- **行长度**：每行不超过79个字符（文档字符串/注释不超过72字符）
- **空行**：
  - 顶层函数和类定义之间空两行
  - 类内方法定义之间空一行
- **导入顺序**：
  1. 标准库导入
  2. 相关第三方库导入
  3. 本地应用/库特定导入
  每组导入之间空一行

### 命名约定
- **变量名**：`lowercase_with_underscores`（小写+下划线）
- **函数名**：`lowercase_with_underscores`
- **类名**：`CapitalizedWords`（驼峰式，无下划线）
- **常量名**：`ALL_CAPS_WITH_UNDERSCORES`
- **私有变量**：单下划线前缀`_private_var`
- **避免的名称**：单字符名称（除了计数器和迭代器）

## 2. 代码可读性

### 表达式和语句
- 避免单行复杂的列表推导式或生成器表达式
- 使用括号隐式连接多行内容
- 二元运算符通常放在行末

### 注释
- 文档字符串使用三重双引号`"""Docstring"""`
- 块注释与代码缩进一致，`#`后加一个空格
- 行内注释与代码至少间隔2个空格

## 3. Pythonic编程习惯

### 优雅的Python特性
```python
# 交换变量
a, b = b, a

# 遍历列表同时获取索引
for index, item in enumerate(items):
    pass

# 使用zip并行迭代
for a, b in zip(list_a, list_b):
    pass

# 上下文管理器处理资源
with open('file.txt') as f:
    content = f.read()
```

### 避免反模式
```python
# # 不推荐：用==比较单例（如None）
# if x == None:  # 反模式
# # 推荐：
# if x is None:
#
# # 不推荐：检查容器是否为空
# if len(container) == 0:  # 反模式
# # 推荐：
# if not container:
```

## 4. 类型提示（Python 3.5+）

```python
def greet(name: str) -> str:
    return f"Hello, {name}"

from typing import List, Dict, Optional

def process(items: List[int], config: Optional[Dict[str, float]] = None) -> bool:
    pass
```

## 5. 异常处理

```python
# 捕获特定异常
try:
    file = open('file.txt')
except FileNotFoundError as e:
    print(f"Error: {e}")
else:
    with file:
        content = file.read()
finally:
    print("操作完成")

# 避免裸except
try:
    something()
except:  # 不推荐 - 会捕获所有异常包括SystemExit和KeyboardInterrupt
    pass

# 推荐
try:
    something()
except Exception:  # 明确捕获Exception
    pass
```

## 6. 函数设计原则

- 函数应该小而专注（单一职责原则）
- 避免函数有太多参数（通常不超过5个）
- 使用默认参数简化调用
- 返回一致的类型（不要有时返回None有时返回其他类型）

## 7. 类设计原则

- 遵循SOLID原则
- 使用属性(@property)而不是直接暴露实例变量
- 优先使用组合而非继承
- 为类和方法编写文档字符串

## 8. 工具支持

- **linter**：flake8, pylint
- **格式化工具**：black, autopep8, yapf
- **类型检查**：mypy
- **IDE支持**：PyCharm, VS Code等都有风格检查功能

良好的Python编程风格不仅能提高代码的可读性和可维护性，还能体现Python"优雅"、"明确"、"简单"的设计哲学。遵循这些准则将使你的代码更Pythonic，更容易被其他Python开发者理解和接受。